In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

RANDOM_SEED = 20090425

---

# Dealing with Missing Data

Missing data is a common problem in most real-world scientific datasets. While the best way for dealing with missing data will always be preventing their occurrence in the first place, it usually can't be helped, particularly when data are collected passively or voluntarily, or when data collection and recording is distributed among several people. 

For example, consider a survey dataset for some wildlife species:

| Count | Site | Observer | Temperature |
| ----- | ---- | -------- | ----------- |
| 15    | 1    | 1        | 15          |
| 10    | 1    | 2        | NA          |
| 6     | 1    | 1        | 11          |

Each row contains the number of individuals seen during the survey, along with three covariates: the site on which the survey was conducted, the observer that collected the data, and the temperature during the survey. If we are interested in modelling, say, population size as a function of the count and the associated covariates, it is difficult to accommodate the second observation because the temperature is missing (perhaps the thermometer was broken that day). Ignoring this observation will allow us to fit the model, but it wastes information that is contained in the other covariates.

There are a variety of ways for dealing with missing data, from the very naïve to the very sophisticated, and unfortunately the more common approaches tend to be *ad hoc* and will usually do more harm than good. 

### Example: Learning outcomes for hearing-impaired children

As a motivating example, we will use a dataset of educational outcomes for children with hearing impairment. Here, we are interested in determining factors that are associated with better or poorer learning outcomes. 

![hearing aid](images/hearing_aid.jpg)

There is a suite of available predictors, including: 

* gender (`male`)
* number of siblings in the household (`siblings`)
* index of family involvement (`family_inv`)
* whether the primary household language is not English (`non_english`)
* presence of a previous disability (`prev_disab`)
* non-white race (`non_white`)
* age at the time of testing (in months, `age_test`)
* whether hearing loss is not severe (`non_severe_hl`)
* whether the subject's mother obtained a high school diploma or better (`mother_hs`)
* whether the hearing impairment was identified by 3 months of age (`early_ident`).

In [ ]:
test_scores = pd.read_csv('../data/test_scores.csv', index_col=0)
test_scores.head()

For three variables in the dataset, there are incomplete records.

In [ ]:
test_scores.isnull().sum(0)

## Strategies for dealing with missing data

The easiest (and worst) way to deal with missing data is to **ignore it**. That is, simply run the analysis, missing values and all, hoping for the best. If your software is any good, this approach will simply not work; the algorithm will try to operate on data that includes missing values, and propagate them, resulting in statistics and estimates that cannot be calculated, which will typically raise errors. If your software is poor, it will make some assumption or decision about the missing values, and proceed to generate  results conditional on the assumption, which creates problems that may never be detected because no indication was given to any potential problem. 

The next easiest (worst) approach to analyzing data with missing values is to conduct list-wise deletion, by deleting the records that have missing values. This is called **complete case analysis**, because only records that are complete get retained for the analysis. The degree to which complete case analysis is undesirable depends on the mechanism by which data have become missing.

## Types of Missingness

- **Missing completely at random (MCAR)**: When data are MCAR, missing cases are, on average, identical to non-missing cases, with respect to the model. Ignoring the missingness will reduce the power of the analysis, but will not bias inference.
- **Missing at random (MAR)**: Missing data depends (usually probabilistically) on measured values, and hence can be modeled by variables observed in the data set. Accounting for the values which “cause” the missing data will produce unbiased results in an analysis.
- **Missing not at random (MNAR)**: Missing data depend on unmeasured or unknown variables. There is no information available to account for the missingness.

> Consider a medical dataset collected that records both age and number of current medications. In this dataset, age is recorded for all individuals, but number of meds is missing for some individuals.
> - if the probability of missingness is independent of both age and meds count, the data are MCAR (*missing at random, observed at random*)
> - if the probability of missingness depends on age but not on meds count, the data are MAR (*missing at random, not observed at random*)

The very best-case scenario for using complete case analysis, which corresponds to MCAR missingness, results in a **loss of power** due to the reduction in sample size. The analysis will lose the information contained in the non-missing elements of a partially-missing record. *Deletion of records due to partially-missing variables will always result in estimates that are inefficient.*

When data are not missing completely at random, inferences from complete case analysis may be **biased** due to systematic differences between missing and non-missing records that affects the estimates of key parameters. Here, we say that the missing data mechanism is **non-ignorable**.

**It is usually better to make up values for missing data than to throw real data away!**

When can record deletion be used?

- Data are missing completely at random
- Fraction of observations with missing elements is very small relative to overall sample size
- predictor and response variables are both absent

If the probability of missingnewss is related to the response variable, estimates can be badly biased. Its important to characterize the missingness in terms of the outcome of interest before deleting observations.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12,5
                                       ))
for i,variable in enumerate(['mother_hs', 'prev_disab', 'family_inv']):
    (test_scores.assign(missing=test_scores[variable].isnull())
         .groupby('missing')
         .score.hist(ax=axes[i]))

## Imputation

To make our estimates as efficient as possible, **we want to preserve the information and interpretation of the non-missing data**.

One alternative to complete case analysis is to simply fill (*impute*) the missing values with a reasonable guess a the true value, such as the mean, median or modal value of the fully-observed records. This imputation, while not recovering any information regarding the missing value itself for use in the analysis, does provide a mechanism for including the non-missing values in the analysis, thereby making use of all available information.

Performing mean imputation is easy in Pandas, via the DataFrame/Series `fillna` method.

In [ ]:
test_scores.siblings.mean()

In [ ]:
siblings_imputed = test_scores.siblings.fillna(test_scores.siblings.mean())

This approach may be reasonable under the MCAR assumption, but may induce bias under a MAR scenario, whereby missing values may **differ systematically** relative to non-missing values, making the particular summary statistic used for imputation *biased* as a mean/median/modal value for the missing values.

Beyond this, the use of a single imputed value to stand in place of the actual missing value glosses over the **uncertainty** associated with this guess at the true value. Any subsequent analysis procedure (*e.g.* regression analysis) will behave as if the imputed value were observed, despite the fact that we are actually unsure of the actual value for the missing variable. The practical consequence of this is that the variance of any estimates resulting from the imputed dataset will be **artificially reduced**.

## Multiple Imputation

One robust alternative to addressing missing data is **multiple imputation** (Schaffer 1999, van Buuren 2012). It produces unbiased parameter estimates, while simultaneously accounting for the uncertainty associated with imputing missing values. It is conceptually and mechanistically straightforward, and produces complete datasets that may be analyzed using any statistical methodology or software one chooses, as if the data had no missing values to begin with.

Multiple imputation generates imputed values based on a **regression model**. This regression model will help us generate reasonable values, particularly if data are MAR, since it uses information in the dataset that may be informative in predicting what the true value may be. Ideally, we want predictor variables that are **correlated** with the missing variable, and with the mechanism of missingness, if any. For example, one might be able to use test scores from one subject to predict missing test scores from another; or, the probability of income reporting to be missing may vary systematically according to the education level of the individual.

To see if there is any potential information among the variables in our dataset to use for imputation, it is helpful to calculate the pairwise correlation between all the variables. Since we have discrete variables in our data, the [Spearman rank correlation coefficient](http://www.wikiwand.com/en/Spearman%27s_rank_correlation_coefficient) is appropriate.

In [ ]:
test_scores.dropna().corr(method='spearman')

We will try to impute missing values the mother's high school education indicator variable, which takes values of 0 for no high school diploma, or 1 for high school diploma or greater. The appropriate model to predict binary variables is a **logistic regression**. 

To keep things simple, we will only use variables that are themselves complete to build the predictive model, hence our subset of predictors will exclude family involvement score (`family_inv`) and previous disability (`prev_disab`).

In [ ]:
impute_subset = test_scores.drop(labels=['family_inv','prev_disab','score'], axis=1)

In [ ]:
y = impute_subset.pop('mother_hs').values
X = impute_subset.astype(float)

The *training* and *test* sets in this case will be the non-missing and missing values, respectively, since we want to use supervised learning to build our predictive model.

In [ ]:
missing = np.isnan(y)

Next, we create a logistic regression model, and fit it using the non-missing observations.

As a performance hack, we will normalize the input variables (center them and subtract by their respective standard deviations). This will make the models run faster.

In [ ]:
normalize = lambda x: (x - x.mean()) / x.std()
X = X.apply(normalize)

Now, we can set up our logistic regression model in PyMC3, as we have done previously.

In [ ]:
from pymc3 import Model, Normal, Bernoulli, invlogit

with Model() as model:

    μ = Normal('μ', 0, sd=10)
    β = Normal('β', 0, sd=10, shape=X.shape[1])

    p = invlogit(μ + β.dot(X[~missing].T))

    mother_hs = Bernoulli('mother_hs', p, observed=y[~missing])

Notice that we have indexed out the non-missing values from `X` and `y`. 

We need to add one additional component to our model: **predictions for the unobserved values**. We can do this by creating a `Deterministic` node that calculates the expected probability at the missing values for high school education:

In [ ]:
from pymc3 import Deterministic

with model:
    
    p_pred = Deterministic('p_pred', 
                    invlogit(μ + β.dot(X.loc[missing].T)))

This generates samples from the **posterior predictive distribution** for `p`, the same mechanism we used when peforming model checking.

We can now run our model and extract the predictions.

In [ ]:
from pymc3 import fit

with model:
    
    tr = fit(20000, random_seed=RANDOM_SEED).sample(500)

In [ ]:
tr['p_pred'].mean(0)

These are *probabilities* of having a high school education; we need to use these in conjunction with a classification rule in order to create imputed values. We will assume `mother_hs=1` if the probability exceeds 0.5, or `mother_hs=0` otherwise.

In [ ]:
(tr['p_pred'].mean(0) > 0.5).astype(int)

These values can then be inserted in place of the missing values, and an analysis can be performed on the entire dataset.

However, this is still just a single imputation for each missing value, and hence glosses over the uncertainty associated with the derivation of the imputes. Multiple imputation proceeds by **imputing several values**, to generate several complete datasets and performing the same analysis on all of them. With a set of estimates in hand, an *average* estimate of model parameters can be obtained that more adequately accounts for the uncertainty, hopefully providing more robust inference than from a single impute.

There are a variety of ways to generate multiple imputations. Here, we will define a *set* of predictive models. For example, we can pick arbitrary subsets of the predictor variables and use each subset to generate a vector of imputed values.

For efficiency, we will wrap a general model specification in a function, which can be instantiated with any subset of covariatest that we specify.

In [ ]:
def imputation_model(predictors):

    with Model() as model:

        μ = Normal('μ', 0, sd=10)
        β = Normal('β', 0, sd=10, shape=len(predictors))

        p = invlogit(μ + β.dot(X.loc[~missing, predictors].T))

        mother_hs = Bernoulli('mother_hs', p, observed=y[~missing])

        p_pred = Deterministic('p_pred', 
                        invlogit(μ + β.dot(X.loc[missing, predictors].T)))
        
    return model

Surprisingly few imputations are required to acheive reasonable estimates, with 3-10 usually sufficient. We will use 3.

In [ ]:
predictors = (['non_english', 'age_test', 'non_severe_hl'],
              ['male', 'siblings', 'non_english'],
              X.columns)

Let's run each of these models in turn and save the imputed values.

In [ ]:
mother_hs_imp = []

for P in predictors:
    
    mod = imputation_model(P)
    
    with mod:
    
        tr = fit(20000, random_seed=RANDOM_SEED).sample(500)

    imputed = (tr['p_pred'].mean(0) > 0.5).astype(int)
    
    mother_hs_imp.append(imputed)

In [ ]:
mother_hs_imp

Now we can perform 3 separate analyses, using the method of our choice, each based upon a different set of imputed values. 

In [ ]:
from pymc3 import HalfCauchy

coefficients = []

for imputes in mother_hs_imp:
        
    X = test_scores.drop(labels=['family_inv','prev_disab'], axis=1).astype(float)
    X.loc[missing, 'mother_hs'] = imputes
    y = X.pop('score')
    
    with Model():

        μ = Normal('μ', 0, sd=10)
        β = Normal('β', 0, sd=10, shape=X.shape[1])
        
        σ = HalfCauchy('σ', 1)

        score = Normal('score', μ + β.dot(X.apply(normalize).T), sd=σ, observed=y)
        
        samples = fit(20000, random_seed=RANDOM_SEED).sample(1000)

    coefficients.append(samples['β'].mean(0))

In [ ]:
coeff_labels = ['male',
                'siblings',
                'non_english',
                'age_test',
                'non_severe_hl',
                'mother_hs',
                'early_ident',
                'non_white']

coef_df = pd.DataFrame(coefficients, columns=coeff_labels)
coef_df

Inference for each coefficient is then based on the average of these three estimates.

In [ ]:
coef_df.mean()

We can see that we obtain somewhat different estimates when we use multiple imputation versus complete case analysis:

In [ ]:
X = test_scores.drop(labels=['family_inv','prev_disab'], axis=1).astype(float).dropna(0)
y = X.pop('score')

with Model():

    μ = Normal('μ', 0, sd=10)
    β = Normal('β', 0, sd=10, shape=X.shape[1])

    σ = HalfCauchy('σ', 1)

    score = Normal('score', μ + β.dot(X.apply(normalize).T), sd=σ, observed=y)

    samples = fit(20000, random_seed=RANDOM_SEED).sample(1000)
    
pd.Series(samples['β'].mean(0), index=coeff_labels)

Estimates from mean imputation are more similar.

In [ ]:
X = test_scores.drop(labels=['family_inv','prev_disab'], axis=1).astype(float)
X['mother_hs'] = X.mother_hs.fillna(X.mother_hs.mean())
y = X.pop('score')

with Model():

    μ = Normal('μ', 0, sd=10)
    β = Normal('β', 0, sd=10, shape=X.shape[1])

    σ = HalfCauchy('σ', 1)

    score = Normal('score', μ + β.dot(X.apply(normalize).T), sd=σ, observed=y)

    samples = fit(20000, random_seed=RANDOM_SEED).sample(1000)
    
pd.Series(samples['β'].mean(0), index=coeff_labels)

### Integrated missing data imputation

A more "fully Bayesian" approach for multiple imputation is to treat the missing values as latent variables, and estimate them just as one would with any other model unknown. This allows imputation to occur within the same modeling framework as the regession analysis itself. However, it involves constructing a sub-model for the missing data, which requires a little additional work.

Let's first extract `mother_hs` from the dataset:

In [ ]:
X = test_scores.drop(labels=['family_inv','prev_disab'], axis=1).astype(float)
y = X.pop('score')
mother_hs_ = X.pop('mother_hs')

If we provide a model for `mother_hs`, PyMC will automatically impute values as it draws posterior samples.

In [ ]:
mother_hs_.head(10)

For simplicity, here we will assume `mother_hs` is MCAR. So, all that is required is a sampling distribution for the variable, which is binary. So, an appropriate model is the Bernoulli distribution, just as we used for the likelihood in the logistic regression model.

We will assign a uniform prior to the probability of the mother having at least a high school education. Here is the complete model:

In [ ]:
from pymc3 import Uniform, sample

with Model() as bayes_impute:
    
    π = Uniform('π', 0, 1)
    mother_hs = Bernoulli('mother_hs', π, observed=mother_hs_)
    
    μ = Normal('μ', 0, sd=10)
    β = Normal('β', 0, sd=10, shape=X.shape[1])
    γ = Normal('γ', 0, sd=10)

    σ = HalfCauchy('σ', 1)

    score = Normal('score', μ + β.dot(X.apply(normalize).T) + γ*mother_hs, sd=σ, observed=y)

    samples = sample(1000, tune=1000)

In [ ]:
samples['mother_hs_imputed_missing'].mean(0)

The MCAR assumption can be relaxed and replaced by a more sophisticated model than the simpl Bernoulli sampling distribtion used here. In fact, we could implement the full multivariate model that we used for multiple imputation relatively easily.

## Capstone Project

To consolidaate what we've learned here, let's consider the problem of predicting survival in the Titanic disaster, based on our available information. Here, we want to predict survival as a function of several variables that are available from the ship's record.

![titanic](images/titanic_cube.png)

In [ ]:
titanic = pd.read_excel("../data/titanic.xls", "titanic")
titanic.name

Fit a reasonable Bayesian multivariate generalized linear model with this data, using several variables such as sex, age and fare to predict survival. Make sure you involve the following steps:

- data imputation
- model checking
- model comparison

In [ ]:
# Write your answer here

## References

- R. J. A. Little and D. B. Rubin. Statistical Analysis with Missing Data. Wiley, New York, second edition, 2002.
- M. J. Knol, K. J. M. Janssen, R. T. Donders, A. C. G. Egberts, E. R. Heerding, D. E. Grobbee, K. G. M. Moons, and M. I. Geerlings. [Unpredictable bias when using the missing indicator method or complete case analysis for missing confounder values: an empirical example](https://www.ncbi.nlm.nih.gov/pubmed/20346625). J Clin Epi, 63:728–736, 2010.
- van Buuren, S. (2012). Flexible Imputation of Missing Data (pp. 1–326).
- Schafer, J. L. (1999). Multiple imputation: a primer. Statistical Methods in Medical Research, 8(1), 3–15. http://doi.org/10.1177/096228029900800102